## Map Stationary and Mobile Sensor Data

In [7]:
import folium
import numpy as np
from constants import Constants

constants = Constants()
map_dir = './'

In [8]:
##Map the stationary sensors
stationary_map = folium.Map(location=[55.943, -3.19], zoom_start=15,tiles="Stamen Toner")
mapStationarySensors(stationary_map)

stationary_map.save(map_dir + "stationarySensors.html")

In [5]:
## Map a data pollution collection walk
mobile_map = folium.Map(location=[55.943, -3.19], zoom_start=15,tiles="Stamen Toner")
sid = 'XXM008' #or 'XXM007'. ID of the pollution belt that data was collected with
mapPollutionWalk(mobile_map, start_date, sid)

#mobile_map.save(map_dir + str(start_date) + "_" + str(sid_plot) + ".html")

NameError: name 'start_date' is not defined

In [3]:
def mapStationarySensors(map_obj):
    staticCoords = constants.getStaticCoords()
    for i in range(staticCoords.shape[0]):
        folium.CircleMarker((staticCoords[i][0], staticCoords[i][1]),
                   radius=8,
                    color='#000000',
                    weight=1.0,
                    fill_color='#000000',
                    fill=True,
                    fill_opacity=1,
                   ).add_to(map_obj)

def mapPollutionWalk(mapObj, start_date, sid):
    end_date = getEndDate(start_date)
    data_dir = "/Users/ryanegan/Documents/diss/projectZoe/data/raw/personal/"+str(start_date)+"-"+str(end_date)+"/"
    sids = ['XXM007', 'XXM008']
    pdata = data_downloader.readAirSpeckPCSV(start_date, end_date, data_dir)
    belt_index = sids.index(sid)
    
    maxPM = np.max(pdata[belt_index]["PM2.5"])
    minPM = np.min(pdata[belt_index]["PM2.5"])
            
    ##Add validation walk
    for j in range(len(pdata[belt_index])):
        folium.CircleMarker((pdata[belt_index]["latitude"][j], pdata[belt_index]["longitude"][j]),
                    radius=5,
                    color='#000000',
                    weight=1.0,
                    fill_color=osm_reader.assignColor(pdata[belt_index]["PM2.5"][j], maxPM, minPM),
                    fill=True,
                    fill_opacity=1.0,
                   ).add_to(mapObj)

def getEndDate(start_date):
    return start_date + 1